# Прогнозирование продаж

Компания хочет понимать, какой клиент из базы данных с большой долей вероятности согласится купить предложенное оборудование. 

В нашем доступе датасет, собранный для случайного множества клиентов ('id' – идентификатор клиента), с которыми была попытка коммуникации в одном из каналов ('channel_name').

Качество сделанных предсказаний оценивается по метрике ROC AUC между истинными значениями и значениями, полученными в ходе исследования.

# Цели исследования

Разработка модели склонности (бинарной классификации) к покупке клиентом оборудования после коммуникации с ним в одном из каналов.

# Описание данных

Датасет собран для случайного множества клиентов (`id` – идентификатор клиента), с которыми была попытка коммуникации в одном из каналов (`channel_name`).<br>
Целевая переменная (`target`) равна единице, если после коммуникации с клиентом была продажа
оборудования и нулю если нет.<br>
Поле `period` соответствует месяцу сбора признаков на клиента. Лаг между датой коммуникации и сборкой признаков на клиента.

Файлы: 
- dataset_train.parquet - тренировочный датасет;
- features_oot.parquet - тестовый датасет;
- features_types.json - описание типов признаков;
- *sample_submission.csv *- пример файла с загружаемыми результатами;

Для каждой пары 'id' + 'period' собрано более 2500 признаков.

Названия признаков интерпретируются следующим образом:

```
    <модуль><номер признака><глубина агрегации>_<тип>

```
Если признак построен как агрегат (например сумма за период), то указывается `<глубина агрегации>` в
месяцах, в противном случае ставится 0. Также, в качестве `<глубина агрегации>` может быть запись вида '3d6', что указывает на отношение агрегата за 3 месяца к агрегату за 6 месяцев.

Различные типы признаков (<тип>) описаны ниже:

- flg - флаг (значение 1 или 0)
- ctg - категориальный признак
- num - числовой признак
- dt - дата
- cnt -количество
- sum -сумма
- avg - среднее
- sumpct -персентиль по сумме
- part - доля

В файле `features_types.json` дополнительно записан словарь, где для каждой фичи в соответствие ставится тип из списка (numeric, categorical_int, categorical_string)

# План работы

1. Изучить информацию о данных и подготовить данные;
2. Провести исследовательский анализ данных;
3. Построить и обучить модель. 
4. Проверить качество лучшей модели на тестовой сборке.

## Обзор и предобработка данных

**Импортируем необходимые библиотеки вначале документа (pandas, numpy, matplotlib.pyplot и другие).**

In [1]:
# задаем константы
RANDOM_STATE = 50623

In [2]:
# импорт библиотеки
import warnings
import time
import random

In [3]:
# настраиваем фильтр предупреждений
warnings.filterwarnings("ignore")

Установка библиотек

!pip install pandas==1.16.5

In [4]:
!pip install --upgrade pyarrow

In [5]:
!pip install --upgrade pandas scipy

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/b1/67/aca1f6e215d957d24d0a290321f368503305480268f9617bf625243e9dea/pandas-2.1.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 66.4 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.1
    Uninstalling pandas-2.1.1:
      Successfully uninstalled pandas-2.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
beatrix-jupyterlab 2023.814.150030 requires jupyter-server~=1.16, but you have jupyter-server 2.12.1 which is incompatible.
beatrix-jupyte

In [6]:
# импорт библиотек для анализа
import pandas as pd
import numpy as np

In [7]:
# импорт библиотек машинного обучения
import lightgbm
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import (
    train_test_split,
    RandomizedSearchCV,
    GridSearchCV)

### Обзор файла JSON

Откроем файл `json` с названиями колонок и типом переменной.

In [8]:
# сохраняем путь к файлу json
path_json = '/kaggle/input/yapr1-hackaton/features_types.json'

In [9]:
# читаем файл json
df_json = pd.read_json(path_json, orient='index')

In [10]:
# переименовываем столбец
df_json.columns = ['d_type']

In [11]:
# сохраняем названия столбцов в список
all_columns = list(df_json.index)

Количество колоннок разного типа.

In [12]:
# распечатаем распределение колонок по типу данных.
print(df_json['d_type'].value_counts())

d_type
numeric            2607
categorical_int     138
categorical_str      31
Name: count, dtype: int64


In [13]:
# удаляем файл для экономии памяти
del df_json

### Обзор основного датасета

Создадим функцию `data_review` для автоматизации предобработки данных. В качестве параметров она получает список колонок(`columns`) и путь к файлу(`file_path`),а также флаг `is_train`. При значении `True` из датасета отбирается 1/7 строк для ускорения подбора параметров.<br>

In [14]:
# Изменение ограничения на количество выводимых рядов
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 170)

In [15]:
# сохраняем путь к файлу
data_path = '/kaggle/input/yapr1-hackaton/dataset_train.parquet'

In [16]:
# 
def data_review(columns, file_path, is_train=False):
    data_frame = pd.read_parquet(file_path, 
                       columns=columns)
    if is_train == True:
        data_frame = data_frame.sample(int(data_frame.shape[0]/7), random_state=RANDOM_STATE)

    # получаем общую информацию о датасете
    data_frame['channel_name'] = data_frame['channel_name'].astype('int')

    print('Review completed.', f'Shape: {data_frame.shape}', '-'*115, sep='\n')
    # возвращаем датасет
    return data_frame

**Вывод:**
1. Импортировали необходимые библиотеки.
2. Считали данные из parquet-файла.
3. Вывели общую информацию по датафрейму: 
    
    - в датафрейме 2776 столбцов и 702086 строки;
    - в датасете присутствуют пропуски.
    
    
4. Подготовились к этапу 'Предобработка данных'.
 - изменили тип данных некоторых признаков ('channel_name' и др.),

 На этапе предобработки данных предполагается:
 - оценить и принять решение, как обработать пропуски,
 - оценить мультиколлинеарность в признаках,
 - оценить и принять решение об устранении выбросов,
 - объединить таблицы в одну для этапа "Исследовательский анализ данных".


5. Из предварительного обзора видно, что почти все данные были приведены к типу float, в том числе и строковые, а затем Нормализованы или закодированы. Это усложнило возможность вникнуть в суть признаков и логически обработать признаки. Также были признаки, где количество пропусков достигало 90%.

Поэтому весь обзор данных скрыт и предобработка проводится автоматически.

---
---

## Предобработка данных

### Пропуски

В первую очередь избавимся от пропусков, где это целесообразно. Везде, где доля пропусков меньше 75%, они заменялись медианной, вычисленной по `channel_name` и `period`. Остальные пропуски оставим без изменений. Для автоматизации написана функция `fillna_median`, которая получает датасет(`dataset`), список колонок, которые не обрабатываются(`col_ignor`). После функция возвращает обработанный датасет.

In [17]:
# функция для обработки пропусков
def fillna_median(dataset, col_ignor):
    # в цикле перебираем каналы
    for channel in dataset['channel_name'].unique():
        # в цикле перебираем периоды
        for period in dataset['period'].unique():
            # в цикле перебираем столбцы, где менее 75% пропусков
            for column in (dataset.drop(col_ignor, axis=1)
                                 .drop(columns=dataset.columns[round(dataset.isna().mean()*100,2) > 75])
                                 .columns):
                # отбираем строки с пропусками и заменяем медианой
                dataset.loc[(dataset[column].isna()) &
                            (dataset['channel_name']==channel) &
                            (dataset['period']==period), column] = (
                            dataset.loc[(dataset['channel_name']==channel) &
                            (dataset['period']==period), column].median())
    
    print('Filling completed.', f'Shape: {dataset.shape}', '-'*115, sep='\n')
    # возвращаем датасет
    return dataset

---
---

### Мультиколлинеарность

Далее избавимся от мультиколлинеарности признаков. Для этого напишем функцию `drop_corr`. На вход функция получает датасет(`dataset`), список столбцов, которые не обрабатываются(`col_ignor`) и флаг(`debug_info`), который позволяет получать информацию о ходе обработки и вывести матрицу корреляций. Функция удаляет признаки, у которых стандартное отклонение меньше 0.02, т.е. столбцы в которых большинство значений повторяются.<br>
Далее получена матрица корреляций и на её основе удалены столбцы с корреляцией более 0.85. В конце функция возвращает обработанный датасет.

In [18]:
# функция для устранения мульти коллинеарности
def drop_corr(dataset, col_ignor, debug_info=False):
    # вывод отладочной информации
    if debug_info == True:
        # размер датасета
        print(dataset.shape)
        # вывод описательной статистики
        dataset.describe()
    # получение описательной статистики
    stat_1 = dataset.drop(col_ignor, axis=1).describe().T
    # удаление стобцов с низкой дисперсией
    dataset = dataset[list(stat_1[stat_1['std']>0.02].index)+col_ignor]
    # вывод отладочной информации
    if debug_info == True:
        # размер датасета
        print(dataset.shape)
    # получение матрицы корреляций
    corr_1 = dataset.corr(method='spearman')
    # создаем список удаляемых столбцов
    del_col = set()
    # в цикле пербирем столбцы
    for col in corr_1.columns:
        # чтобы не удалить оба столбца, проверяем что столбца нет в списке
        if col not in del_col:
            # удаляем обрабатываемый столбец, чтобы не удалить,
            # берем модуль от корриляции
            temp = corr_1[col].drop(col, axis=0).abs()
            # удаляем столбцы с большой корреляцией
            del_col.update(set((temp[(temp > 0.85)].index)))
    # вывод отладочной информации
    if debug_info == True:
        # список удаляемых столбцов
        print(del_col)
    # удаляем столбцы
    dataset = dataset.drop(del_col.difference(set(col_ignor)), axis=1)
    # вывод отладочной информации
    if debug_info == True:
        # печатаем матрицу корреляций
        display(dataset.corr(method='spearman')
                .style.background_gradient(cmap='Reds'))
        # размер датасета
        print(dataset.shape)
    print('drop_corr completed.', f'Shape: {dataset.shape}', '-'*115, sep='\n')
    return dataset

---
---

### Устранение выбросов

Для устранения выбросов напишем функцию `fill_outliers`, которая получает датасет(`dataset`) и список столбцов, которые не обрабатываются(`col_ignor`). Для каждого столбца вычисляется верхняя(`upper_whiskers`) и нижняя(`lower_whiskers`) границы нормального расрпеделения. Далее все значения, которые находятся за пределами нормального распределения, заполняются ими. В конце функция возвращает обработанный датасет.

In [19]:
# функция для обработки выбросов
def fill_outliers(dataset, col_ignor):
    # в цикле переберем все столбцы, кроме date
    for column in dataset.drop(col_ignor, axis=1).columns:
        Q1 = dataset[column].quantile(0.25) # 1-й квартиль
        Q3 = dataset[column].quantile(0.75) # 3-й квартиль
        IQR = Q3 - Q1 # межквартильный размах
        upper_whiskers = Q3 + 1.5*IQR # верхняя граница
        lower_whiskers = Q1 - 1.5*IQR # нижняя граница
        # избавимся от выбросов
        dataset.loc[dataset[column] < lower_whiskers, column] = \
        lower_whiskers
        dataset.loc[dataset[column] > upper_whiskers, column] = \
        upper_whiskers
    print('fill_outliers completed.', f'Shape: {dataset.shape}', '-'*115, sep='\n')
    # возвращаем датасет
    return dataset

## Обучение модели

### Разделение на выборки

Напишем функцию `data_split` для разделения данных на выборки. На вход она получает датасет(`dataset`) и возвразает датасет с признаками без `id`,`target` и датасет с целевой переменной `target`.

In [20]:
# функция для разделения на выборки
def data_split(dataset):
    # сохраним в features все столбцы, кроме id, period, target
    # сохраним в target целевой признак
    return (dataset.drop(['id', 'target'], axis=1),
           dataset['target'])

Напишем функцию для автоматизации обучения модели. На вход она получает датасет(`dataset`) и флаг `is_test`, от которого зависит набор перебираемых параметров. Внутри вызывается функция `data_split` для разделения на выборки. Для обучения используется `LGBMClassifier` с параметрами `class_weight` для автобаланса веса классов и `device_type='GPU'` для использования `GPU`, используемый тип бустинга(`boosting_type`) `goss`(обуспечивает более быстрое обучение).<br>
Остальные параметры перебираются с помощью модуля `RandomizedSearchCV` с метрикой(`scoring`) `ROC-AUC` и использованием кросс-валидации(`cv`=10). Перебирались следующие гиперпараметры:
* `n_estimators` - количество деревьев в ансамбле.
* `max_depth` - максимальная глубина деревьев в ансамбле.
* `learning_rate` - шаг обучения.
* `num_leaves` - количество листьев.
* `reg_alpha` - коэффициент регуляризации `l1`.
* `reg_beta` - коэффициент регуляризации `l2`.

In [21]:
# функция для обучения моделей
def lgbm_train(dataset, is_test=False):
    # разделение данных на выборки
    X_train, y_train = data_split(dataset)
    # random_state не перебирается, задаём его прямо в модели
    # создание классификатора LightGBM
    model_lgbm = LGBMClassifier(random_state=RANDOM_STATE,
                                class_weight = 'balanced',
                                device_type='GPU',
                                num_gpu=512
                               )

    # словарь с гиперпараметрами и значениями при подборе фичей
    if is_test == False:
        param_grid_lgbm = {
            'boosting_type': ['goss'],
            'n_estimators': [20, 35, 60],
            'max_depth': [2, 4, 7],
            'random_state': [RANDOM_STATE],
            'learning_rate': [0.05, 0.2],
            'force_col_wise': [True],
            'num_leaves': [20, 31, 51, 70],
            'reg_alpha': [0, 0.05, 0.4],
            'reg_lambda': [0, 0.05, 0.4]
        }
    # словарь с гиперпараметрами и значениями при обучении финальной модели
    else:
        param_grid_lgbm = {
            'boosting_type': ['goss'],
            'n_estimators': [50, 70, 100, 120],
            'max_depth': [3, 7, 12],
            'random_state': [RANDOM_STATE],
            'learning_rate': [0.1, 0.02, 0.01],
            'force_col_wise': [True],
            'num_leaves': [30, 50, 71],
            'reg_alpha': [0, 0.2, 0.4, 0.8],
            'reg_lambda': [0, 0.2, 0.4, 0.8]
        }
    # создадим объект GridSearchCV
    rs_lgbm = RandomizedSearchCV(
        model_lgbm, 
        param_distributions=param_grid_lgbm, 
        scoring='roc_auc',
        cv = 10,
        n_jobs=-1
    )
    # обучим модель
    rs_lgbm.fit(X_train, y_train)

    # лучшее значение ROC_AUC на кросс-валидации
    print(f'best_score: {rs_lgbm.best_score_}')

    # лучшие гиперпараметры
    print(f'best_params: {rs_lgbm.best_params_}')
    
    # выводим 5 самых важных признаков
    features_imp = pd.DataFrame(data=rs_lgbm.best_estimator_.feature_name_,
                           columns=['name'])
    features_imp['value'] = list(rs_lgbm.best_estimator_.feature_importances_)
    display(features_imp.sort_values(by='value', ascending=False).head())
    
    values = list(rs_lgbm.best_estimator_.feature_importances_)
    # сохраняем перечень наиболе важных признаков, 
    # порог более importance более 15% от максимума
    imp_features = (pd.Series(rs_lgbm.best_estimator_.feature_name_)
                    [values > 0.15*max(values)].to_list())
    print(imp_features)
    print('Trainning completed.', f'Shape: {dataset.shape}', '-'*115, sep='\n')
    
    # возвращаем список важных признаков
    if is_test == False:
        return imp_features
    else:
        return imp_features

---
---

Далее в цикле перебираем по 200 признаков, добавляя их к отобранным ранее. К каждому набору применяются функции описанные выше. Данный цикл повторяется 3 раза, после перемешивания признаков.

In [22]:
t1 = time.perf_counter()
# создаем множество для хранения признаков
imp_features = set()
# количество перебираемых признаков
batch_size = 200

for i in range(3):
    # перебираем по 200 признаков + отобранные ранее
    for i in range(0, (2775 - batch_size + 3), batch_size):
        temp_columns = (list(imp_features.union(['id', 'period', 'channel_name'] +
                                                all_columns[i : i + batch_size] + ['target']))
             )
        
        # печатаем номера обрабатываемых признаков
        print(f'Dataset_{int(1+i/batch_size)}\ncolumns: {i+1}:{i+batch_size+1}')
        # создаем датафрейм
        temp_df = pd.DataFrame()
        # сохраняем данные в датафрейм
        temp_df = data_review(temp_columns, data_path, is_train=True)
        # устраняем мулти коллинеарность
        temp_df = drop_corr(temp_df, ['id', 'period', 'target'], debug_info=False)
        #print('Shape: ', temp_df.shape)
        # заполняем пропуски
        temp_df = fillna_median(temp_df, ['id', 'period', 'target'])
        # устраняем выбросы
        temp_df = fill_outliers(temp_df, ['id', 'period', 'target', 'channel_name'])
        #print('Shape: ', temp_df.shape)
        
        # метка времени перед началом обучения
        t2 = time.perf_counter()
        # обучаем модель и сохраняем отобранные признаки
        imp_features = imp_features.union(set(lgbm_train(temp_df)))
        # метка после обучения
        t3 = time.perf_counter()
        print(f'fit time: {t3-t2:.4f} s')
        # удаляем не используемую переменную
        del temp_df
        print('-'*345)
    # перемешиваем список столбцов
    random.shuffle(all_columns)
    
print(f'Total time: {t3 - t1}')

Dataset_1
columns: 1:201
Review completed.
Shape: (100298, 204)
-------------------------------------------------------------------------------------------------------------------
drop_corr completed.
Shape: (100298, 201)
-------------------------------------------------------------------------------------------------------------------
Filling completed.
Shape: (100298, 201)
-------------------------------------------------------------------------------------------------------------------
fill_outliers completed.
Shape: (100298, 201)
-------------------------------------------------------------------------------------------------------------------


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


best_score: 0.6943171500102052
best_params: {'reg_lambda': 0, 'reg_alpha': 0, 'random_state': 50623, 'num_leaves': 31, 'n_estimators': 35, 'max_depth': 2, 'learning_rate': 0.2, 'force_col_wise': True, 'boosting_type': 'goss'}


,name,value
158,channel_name,24
120,markers_4_1_cnt,9
133,markers_184_1_cnt,8
4,markers_40_1_cnt,8
0,markers_60_1_cnt,6


['markers_60_1_cnt', 'markers_40_1_cnt', 'markers_72_1_cnt', 'markers_122_1_cnt', 'markers_104_1_cnt', 'markers_199_1_cnt', 'markers_74_1_cnt', 'markers_4_1_cnt', 'markers_184_1_cnt', 'channel_name', 'markers_59_1_cnt', 'markers_135_1_cnt', 'period']
Trainning completed.
Shape: (100298, 201)
-------------------------------------------------------------------------------------------------------------------
fit time: 100.9015 s
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Dataset_2
columns: 201:401
Review completed.
Shape: (100298, 215)
-------------------------------------------------------------------------------------------------------------------
drop_corr completed.
Shape: (100298, 210)


...

['charges_details_6_1_sum', 'charges_details_5_6_avg', 'info_house_6_0_num', 'spas_symptoms_agr_154_12_sum', 'markers_508_1_cnt', 'traffic_details_53_1_sum', 'materials_details_16_1_ctg', 'communication_availability_30_1_flg', 'campaigns_394_3d6_part', 'channel_name']
Trainning completed.
Shape: (100298, 212)
-------------------------------------------------------------------------------------------------------------------
fit time: 142.4349 s
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Dataset_12
columns: 2201:2401
Review completed.
Shape: (100298, 277)
-------------------------------------------------------------------------------------------------------------------
drop_corr completed.


,name,value
208,channel_name,17
168,materials_details_16_1_ctg,6
175,communication_availability_30_1_flg,5
14,charges_details_6_1_sum,4
43,info_house_6_0_num,4


['charges_details_6_1_sum', 'charges_details_5_6_avg', 'traffic_details_16_1d3_std', 'spas_symptoms_agr_105_12_std', 'info_house_6_0_num', 'campaigns_400_1d6_part', 'markers_508_1_cnt', 'traffic_details_53_1_sum', 'materials_details_16_1_ctg', 'communication_availability_30_1_flg', 'campaigns_394_3d6_part', 'payments_details_48_3_sum', 'channel_name']
Trainning completed.
Shape: (100298, 224)
-------------------------------------------------------------------------------------------------------------------
fit time: 137.3392 s
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Dataset_13
columns: 2401:2601
Review completed.
Shape: (100298, 277)
----------------------------------------------------

,name,value
197,channel_name,13
168,communication_availability_30_1_flg,4
158,materials_details_16_1_ctg,4
13,charges_details_6_1_sum,3
127,traffic_details_53_1_sum,3


['charges_details_6_1_sum', 'charges_details_5_6_avg', 'movix_channels_107_6_sumpct', 'spas_symptoms_agr_105_12_std', 'traffic_details_65_3_sum', 'charges_details_14_6_avg', 'markers_508_1_cnt', 'traffic_details_53_1_sum', 'materials_details_16_1_ctg', 'communication_availability_30_1_flg', 'channel_name']
Trainning completed.
Shape: (100298, 215)
-------------------------------------------------------------------------------------------------------------------
fit time: 127.8297 s
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Total time: 12732.909994644


**Вывод:**
1. Подобрали признаки для основного расчета.
2. Лучшая метрика 0,749.
3. Параметры при которых получена наилучшая метрика: 
    
    - 'reg_lambda': 0.4, 
    - 'reg_alpha': 0, 
    - 'random_state': 50623, 
    - 'num_leaves': 51, 
    - 'n_estimators': 60, 
    - 'max_depth': 2, 
    - 'learning_rate': 0.2, 
    - 'force_col_wise': True, 
    - 'boosting_type': 'goss'
    

---
---

### Обучение итоговой модели

Сохраним в `final_columns` список отобранных признаков.

In [23]:
final_columns = list(set(imp_features).union(['id', 'period', 'channel_name', 'target']))

Создаем датафрейм для обучения финального датасета.

In [24]:
final_df = pd.DataFrame()

Сохраняем данные в датафрейм.

In [25]:
final_df = data_review(final_columns, data_path)

Review completed.
Shape: (702086, 82)
-------------------------------------------------------------------------------------------------------------------


Заполняем пропуски.

In [26]:
final_df = fillna_median(final_df, ['id', 'period', 'target'])

Filling completed.
Shape: (702086, 82)
-------------------------------------------------------------------------------------------------------------------


Устраняем выбросы.

In [27]:
final_df = fill_outliers(final_df, ['id', 'period', 'target', 'channel_name'])

fill_outliers completed.
Shape: (702086, 82)
-------------------------------------------------------------------------------------------------------------------


In [28]:
print(final_df.shape)

(702086, 82)


Избавляемся от мультиколлинеарности.

In [29]:
final_df = drop_corr(final_df, ['id', 'period', 'target'], debug_info=True)

(702086, 82)
(702086, 81)
{'spas_symptoms_agr_105_12_std', 'basic_info_0_0_avg', 'movix_channels_107_6_sumpct', 'campaigns_392_3_part', 'traffic_details_21_3_avg', 'campaigns_397_1_part', 'smarttv_age_1_1_avg', 'campaigns_2_6_cnt', 'spas_symptoms_agr_106_12_sum'}


,spas_symptoms_agr_104_12_avg,markers_60_1_cnt,tariff_plans_4_1_num,markers_534_1_cnt,markers_434_1_cnt,markers_40_1_cnt,tariff_plans_19_src_id,spas_symptoms_int_104_1_cnt,communication_availability_30_1_flg,materials_details_21_1_num,smarttv_age_2_1_max,campaigns_400_1d6_part,markers_508_1_cnt,traffic_details_15_1d3_avg,materials_details_4_1_dt,spas_symptoms_agr_9_12_std,markers_72_1_cnt,issues_138_3d6_sum,autopay_7_0_dt,charges_details_6_1_sum,markers_346_1_cnt,markers_542_1_cnt,markers_74_1_cnt,user_active_23_0_dt,charges_details_5_6_avg,markers_4_1_cnt,spas_symptoms_agr_76_3_sum,movix_channels_104_6_sum,traffic_details_53_1_sum,campaigns_395_6_part,campaigns_394_3d6_part,markers_533_1_cnt,markers_333_1_cnt,traffic_details_36_3_avg,markers_122_1_cnt,markers_537_1_cnt,markers_380_1_cnt,markers_184_1_cnt,markers_310_1_cnt,campaigns_378_6_cnt,spas_symptoms_agr_151_6_sum,traffic_details_16_1d3_std,markers_232_1_cnt,basic_info_2_0_min,user_active_22_0_dt,user_lifetime_2_1_num,traffic_details_26_3d6_sum,spas_symptoms_agr_163_6_sum,markers_104_1_cnt,payments_details_48_3_sum,info_house_6_0_num,markers_334_1_cnt,channel_name,tariff_plans_21_1_max,spas_symptoms_agr_154_12_sum,traffic_details_19_1d6_std,traffic_details_62_1_sum,payments_details_49_6_avg,charges_details_14_6_avg,markers_59_1_cnt,markers_199_1_cnt,info_house_5_0_num,markers_135_1_cnt,materials_details_16_1_ctg,campaigns_36_3_part,markers_349_1_cnt,user_active_25_0_dt,markers_330_1_cnt,markers_324_1_cnt,id,period,target
spas_symptoms_agr_104_12_avg,1.000000,-0.044407,-0.085054,-0.081299,-0.035424,0.028535,-0.111246,0.011888,-0.155137,0.107906,0.016417,0.014980,-0.006296,-0.008355,0.279019,-0.063477,-0.058398,0.000422,0.035498,-0.098110,-0.006934,-0.065912,-0.071640,0.104492,0.086678,-0.098840,-0.026337,0.067673,0.045352,-0.034126,0.020474,-0.121657,-0.038617,-0.096426,-0.035147,-0.092121,-0.035095,-0.042700,-0.010641,0.134689,-0.022868,0.007559,-0.092683,0.071448,0.156614,0.192580,-0.052570,-0.156516,-0.107843,0.056233,0.075491,-0.040859,-0.028229,-0.095447,-0.184526,0.019064,0.030830,0.032458,0.095285,-0.044875,-0.067657,-0.093955,-0.015726,-0.425579,-0.008034,-0.060091,0.049277,-0.064969,-0.062904,0.000796,-0.123025,0.028512
markers_60_1_cnt,-0.044407,1.000000,0.123458,0.126484,0.026355,0.138903,0.043162,-0.014296,0.088623,0.028423,0.004998,0.009827,0.067584,0.027908,0.007287,0.049652,0.307568,0.008521,0.029423,0.082822,0.235494,0.134271,0.323115,0.051615,0.014751,0.229825,-0.014575,-0.048860,0.142457,0.074879,0.004020,0.151933,0.235213,0.268866,0.267219,0.143137,0.178889,0.171381,0.257278,0.020075,0.035711,0.025551,0.196532,-0.047319,0.012963,0.031377,0.019600,-0.017218,0.151196,0.019480,-0.014887,0.154359,-0.078659,0.087934,-0.003569,0.037658,0.209118,0.013960,-0.059168,0.055538,0.257547,0.047538,0.110339,0.054478,0.118680,0.278371,0.045290,0.293838,0.129568,0.000334,0.039023,-0.003487
tariff_plans_4_1_num,-0.085054,0.123458,1.000000,0.076100,-0.007693,-0.015119,0.338774,-0.116430,0.080653,0.139087,0.060759,-0.013762,-0.027055,0.043673,-0.015831,-0.040790,0.106805,0.019190,-0.000741,0.300124,0.010784,0.009567,0.107642,0.062691,0.346169,0.051321,0.058524,-0.003306,-0.138131,0.028886,-0.028531,0.078399,0.047403,0.182512,0.090065,0.072875,-0.000789,0.006211,0.050959,-0.159853,0.154043,0.043674,0.067401,-0.058922,-0.039580,0.000982,0.038001,-0.127534,-0.004248,0.286980,-0.162783,0.070101,0.013402,0.415029,-0.136578,0.072060,-0.085871,0.277267,-0.216425,0.048440,0.093574,0.217000,0.013463,0.104590,0.056600,0.078355,0.109760,0.078691,0.017754,0.001790,0.149010,-0.008257
markers_534_1_cnt,-0.081299,0.126484,0.076100,1.000000,0.182784,0.146949,0.075011,-0.056988,0.065274,-0.016142,-0.010844,0.026471,0.140009,0.038085,-0.034704,0.091302,0.228530,-0.001279,-0.029663,0.010768,0.146621,0.253408,0.252928,-0.061989,0.030810,0.121521,-0.013955,-0.072472,0.177539,0.130120,-0.013933,0.485580,0.139310,0.271870,0.114520,0.374780,0.194805,0.222188,0.163816,0.043986,0.071123,0.020327,0.

(702086, 72)
drop_corr completed.
Shape: (702086, 72)
-------------------------------------------------------------------------------------------------------------------


In [30]:
print(final_df.shape)

(702086, 72)


Обучаем финальную модель.

In [31]:
# метка времени перед началом обучения
t9 = time.perf_counter()
best_param = lgbm_train(final_df, is_test=True)
# метка после обучения
t10 = time.perf_counter()
print(f'fit time: {t10-t9:.4f} s')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


best_score: 0.7459477451806957
best_params: {'reg_lambda': 0.2, 'reg_alpha': 0.2, 'random_state': 50623, 'num_leaves': 71, 'n_estimators': 50, 'max_depth': 12, 'learning_rate': 0.1, 'force_col_wise': True, 'boosting_type': 'goss'}


,name,value
43,basic_info_2_0_min,113
29,campaigns_395_6_part,108
45,user_lifetime_2_1_num,103
61,info_house_5_0_num,101
0,spas_symptoms_agr_104_12_avg,95


['spas_symptoms_agr_104_12_avg', 'markers_60_1_cnt', 'tariff_plans_4_1_num', 'markers_434_1_cnt', 'markers_40_1_cnt', 'tariff_plans_19_src_id', 'spas_symptoms_int_104_1_cnt', 'communication_availability_30_1_flg', 'materials_details_21_1_num', 'campaigns_400_1d6_part', 'markers_508_1_cnt', 'traffic_details_15_1d3_avg', 'materials_details_4_1_dt', 'spas_symptoms_agr_9_12_std', 'issues_138_3d6_sum', 'autopay_7_0_dt', 'charges_details_6_1_sum', 'markers_346_1_cnt', 'markers_542_1_cnt', 'markers_74_1_cnt', 'user_active_23_0_dt', 'charges_details_5_6_avg', 'markers_4_1_cnt', 'movix_channels_104_6_sum', 'traffic_details_53_1_sum', 'campaigns_395_6_part', 'campaigns_394_3d6_part', 'markers_533_1_cnt', 'markers_333_1_cnt', 'traffic_details_36_3_avg', 'markers_122_1_cnt', 'markers_537_1_cnt', 'markers_380_1_cnt', 'markers_184_1_cnt', 'markers_310_1_cnt', 'campaigns_378_6_cnt', 'spas_symptoms_agr_151_6_sum', 'traffic_details_16_1d3_std', 'markers_232_1_cnt', 'basic_info_2_0_min', 'user_lifetime_

## Предсказание на тестовой выборке

Загружаем тестовую выборку.

In [32]:
# Добавляем id и period
df_test_1 = pd.read_parquet('/kaggle/input/yapr1-hackaton/features_oot.parquet', 
                            columns=final_df.drop('target', axis=1).columns)

In [33]:
print(df_test_1.shape)

(60661, 71)


In [34]:
# изменяем тип колонки на int
df_test_1['channel_name'] = df_test_1['channel_name'].astype('int')

Заполняем пропуски.

In [36]:
df_test_1 = fillna_median(df_test_1, ['id', 'period'])

Filling completed.
Shape: (60661, 71)
-------------------------------------------------------------------------------------------------------------------


Устраняем выбросы.

In [37]:
df_test_1 = fill_outliers(df_test_1, ['id', 'period', 'channel_name'])

fill_outliers completed.
Shape: (60661, 71)
-------------------------------------------------------------------------------------------------------------------


In [38]:
print(final_df.shape, df_test_1.shape, sep='\n')

(702086, 72)
(60661, 71)


Список отобранных параметров.

In [39]:
print(final_df.columns.to_list())

['spas_symptoms_agr_104_12_avg', 'markers_60_1_cnt', 'tariff_plans_4_1_num', 'markers_534_1_cnt', 'markers_434_1_cnt', 'markers_40_1_cnt', 'tariff_plans_19_src_id', 'spas_symptoms_int_104_1_cnt', 'communication_availability_30_1_flg', 'materials_details_21_1_num', 'smarttv_age_2_1_max', 'campaigns_400_1d6_part', 'markers_508_1_cnt', 'traffic_details_15_1d3_avg', 'materials_details_4_1_dt', 'spas_symptoms_agr_9_12_std', 'markers_72_1_cnt', 'issues_138_3d6_sum', 'autopay_7_0_dt', 'charges_details_6_1_sum', 'markers_346_1_cnt', 'markers_542_1_cnt', 'markers_74_1_cnt', 'user_active_23_0_dt', 'charges_details_5_6_avg', 'markers_4_1_cnt', 'spas_symptoms_agr_76_3_sum', 'movix_channels_104_6_sum', 'traffic_details_53_1_sum', 'campaigns_395_6_part', 'campaigns_394_3d6_part', 'markers_533_1_cnt', 'markers_333_1_cnt', 'traffic_details_36_3_avg', 'markers_122_1_cnt', 'markers_537_1_cnt', 'markers_380_1_cnt', 'markers_184_1_cnt', 'markers_310_1_cnt', 'campaigns_378_6_cnt', 'spas_symptoms_agr_151_6_

Функция для разделения на выборки.

In [40]:
def data_split_2(dataset):
    # сохраним в features все столбцы, кроме id, period, target
    # сохраним в target целевой признак
    return (dataset.drop(['id', 'target'], axis=1),
           dataset['target'])

Создаем тестовую выборку.

In [41]:
X_test = df_test_1.drop(['id'], axis=1)

Обучаем финальную модель.

In [42]:
t9 = time.perf_counter()

X_train, y_train = data_split_2(final_df)
# random_state не перебирается, задаём его прямо в модели
model_lgbm = LGBMClassifier(verbose=-1, random_state=RANDOM_STATE,
                            class_weight = 'balanced',
                            #scale_pos_weight=127,
                            device_type='GPU',
                            num_gpu=512
                            )

# словарь с гиперпараметрами и значениями, которые хотим перебрать
param_grid_lgbm = {
    'boosting_type': ['goss'],
    'n_estimators': [30, 50, 70],
    'max_depth': [2, 5, 7, 12],
    'random_state': [RANDOM_STATE],
    'learning_rate': [0.1, 0.05, 0.02, 0.01],
    'force_col_wise': [True],
    'num_leaves': [20, 30, 50, 71],
    'reg_alpha': [0, 0.05, 0.3, 0.8],
    'reg_lambda': [0, 0.05, 0.3, 0.8]
}
# создадим объект GridSearchCV
gs_lgbm = RandomizedSearchCV(
    model_lgbm, 
    param_distributions=param_grid_lgbm, 
    scoring='roc_auc',
    cv = 10,
    n_jobs=-1
)
# обучим модель
gs_lgbm.fit(X_train, y_train)
t10 = time.perf_counter()
# лучшее значение ROC_AUC на кросс-валидации
print(f'best_score: {gs_lgbm.best_score_}')

# лучшие гиперпараметры
print(f'best_params: {gs_lgbm.best_params_}')
print(f'fit time: {t9-t10:.4f} s')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


best_score: 0.7498232741346947
best_params: {'reg_lambda': 0, 'reg_alpha': 0, 'random_state': 50623, 'num_leaves': 71, 'n_estimators': 70, 'max_depth': 5, 'learning_rate': 0.1, 'force_col_wise': True, 'boosting_type': 'goss'}
fit time: -418.4474 s


In [43]:
# лучшее значение ROC_AUC на кросс-валидации
print(f'best_score: {gs_lgbm.best_score_}')
features_imp = pd.DataFrame(data=gs_lgbm.best_estimator_.feature_name_,
                           columns=['name'])
features_imp['value'] = list(gs_lgbm.best_estimator_.feature_importances_)
display(features_imp.sort_values(by='value', ascending=False))

best_score: 0.7498232741346947


,name,value
52,channel_name,100
24,charges_details_5_6_avg,74
45,user_lifetime_2_1_num,72
43,basic_info_2_0_min,69
0,spas_symptoms_agr_104_12_avg,65
61,info_house_5_0_num,61
29,campaigns_395_6_part,56
8,communication_availability_30_1_flg,53
56,traffic_details_62_1_sum,52
14,materials_details_4_1_dt,50


Получаем предсказание на тестовой выборке.

In [44]:
test_predict =  gs_lgbm.best_estimator_.predict_proba(X_test)

Считываем файл, куда сохраним результаты.

In [45]:
submission = pd.read_csv('/kaggle/input/yapr1-hackaton/sample_submission.csv')
display(submission)

,id,target
0,0,0.343518
1,1,0.591216
2,2,0.913150
3,3,0.560035
4,4,0.352795
...,...,...
60656,60656,0.765319
60657,60657,0.533016
60658,60658,0.784497
60659,60659,0.804431


In [46]:
#Заменяем столбец с данными из примера на предикт
submission['target'] = test_predict[:,1]
display(submission)

,id,target
0,0,0.560092
1,1,0.481845
2,2,0.540904
3,3,0.573305
4,4,0.653443
...,...,...
60656,60656,0.566511
60657,60657,0.464334
60658,60658,0.682976
60659,60659,0.604190


Созраняем файл с результатми.

In [47]:
#Сохраняем данные на гугл диск или локально и потом сабмитим результат
submission.to_csv('/kaggle/working/my_predict.csv', index=False)

## Отчет по работе

При проверке на публичном датасете самая высокая метрика `ROC-AUC`, которую удалось достичь, 0,6473.
Ее удалось добиться достаточно подробным подбором признаков на модели `LightGBM`. 

Также была опробована модель `Catboost`, которая не дала метрику значительно выше, но по времени расчета была гораздо весомее. 

На скрытой выборке результат получился 0,66. Модель была устойчивой и не потеряла в качестве. 

Очень сильно повлияли на нее значения параметра `'learning_rate'`, а также размер `'iterations'` равный 1000. 

Наиболее удачное количество признаков для расчета это 120-150 признаков. 

Также хотелось бы отметить, что из-за большого файла данных, из которого брались значения признаков и сами признаки - попробовать посчитать модель `Catboost` на GPU на GoogleColab было невозможно, так как время сессии было гораздо меньше, чем скорость загрузки документа на сайт. В Kaggle данный расчет на GPU тоже подвисал после 40 минут расчетов. 

В качестве рекомендаций для улучшения кода: 

- Провести все этапы предобработки через pipeline;
- Подумать над новыми способами предобработки (подбор признаков и заполнение пропусков);
- Попробовать решить данную задачу математически или через  нейронные сети(хотя применение нейронок в отношении табличных данных сомнительно)